<a href="https://colab.research.google.com/github/123shwetarohokale/lammps-aluminum-optimization/blob/main/analyze_energy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Load atomic positions
positions = []
reading_atoms = False

with open("al.data.out") as file:
    for line in file:
        if "Atoms # atomic" in line:
            reading_atoms = True
            continue
        if reading_atoms:
            if line.strip() == "" or line[0].isalpha():
                break  # stop reading if we hit an empty or new section
            parts = line.strip().split()
            x, y, z = float(parts[2]), float(parts[3]), float(parts[4])
            positions.append((x, y, z))

# Convert to numpy array for easier handling
import numpy as np
positions = np.array(positions)

# 3D scatter plot
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(positions[:, 0], positions[:, 1], positions[:, 2], s=5, alpha=0.6)

ax.set_xlabel('X (Å)')
ax.set_ylabel('Y (Å)')
ax.set_zlabel('Z (Å)')
ax.set_title('Aluminum Atomic Structure from LAMMPS')

plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'al.data.out'